In [1]:
import os, gc
import numpy as np
import pandas as pd 
from pathlib import Path
import matplotlib.pyplot as plt
from typing import List, Dict
from tqdm.notebook import tqdm
from time import time, ctime

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import KFold, GroupKFold
from torch.utils.data import DataLoader, Dataset

from engine_hms_trainer import (
    seed_everything, evaluate_oof, get_logger, TARGETS, TARGETS_PRED, Trainer
)

from engine_hms_model import (
    KagglePaths, LocalPaths, ModelConfig
)

from scipy.signal import butter, lfilter, freqz
from scipy.stats import entropy
from scipy.special import rel_entr

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PATHS = KagglePaths if os.path.exists(KagglePaths.OUTPUT_DIR) else LocalPaths
print("Output Dir: ", PATHS.OUTPUT_DIR)

EEG_FEAT_ALL = [
    'Fp1', 'F3', 'C3', 'P3', 
    'F7', 'T3', 'T5', 'O1', 
    'Fz', 'Cz', 'Pz', 'Fp2', 
    'F4', 'C4', 'P4', 'F8', 
    'T4', 'T6', 'O2', 'EKG'
    ]

EEG_FEAT_USE =  ['Fp1','T3','C3','O1','Fp2','C4','T4','O2']
EEF_FEAT_INDEX = {x:y for x,y in zip(EEG_FEAT_USE, range(len(EEG_FEAT_USE)))}

seed_everything(ModelConfig.SEED)

Output Dir:  ./outputs/


In [3]:
def eeg_from_parquet(parquet_path: str, display: bool = False) -> np.ndarray:
    """
    This function reads a parquet file and extracts the middle 50 seconds of readings. Then it fills NaN values
    with the mean value (ignoring NaNs).
    :param parquet_path: path to parquet file.
    :param display: whether to display EEG plots or not.
    :return data: np.array of shape  (time_steps, eeg_features) -> (10_000, 8)
    """
    # === Extract middle 50 seconds ===
    eeg = pd.read_parquet(parquet_path, columns=EEG_FEAT_USE)
    rows = len(eeg)
    offset = (rows - 10_000) // 2 # 50 * 200 = 10_000
    eeg = eeg.iloc[offset:offset+10_000] # middle 50 seconds, has the same amount of readings to left and right
    if display: 
        plt.figure(figsize=(10,5))
        offset = 0
    # === Convert to numpy ===
    data = np.zeros((10_000, len(EEG_FEAT_USE))) # create placeholder of same shape with zeros
    for index, feature in enumerate(EEG_FEAT_USE):
        x = eeg[feature].values.astype('float32') # convert to float32
        mean = np.nanmean(x) # arithmetic mean along the specified axis, ignoring NaNs
        nan_percentage = np.isnan(x).mean() # percentage of NaN values in feature
        # === Fill nan values ===
        if nan_percentage < 1: # if some values are nan, but not all
            x = np.nan_to_num(x, nan=mean)
        else: # if all values are nan
            x[:] = 0
        data[:, index] = x
        if display: 
            if index != 0:
                offset += x.max()
            plt.plot(range(10_000), x-offset, label=feature)
            offset -= x.min()
    if display:
        plt.legend()
        name = parquet_path.split('/')[-1].split('.')[0]
        plt.yticks([])
        plt.title(f'EEG {name}',size=16)
        plt.show()    
    return data

def get_non_overlap(df_csv, targets):
    # Reference Discussion:
    # https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

    tgt_list = targets.tolist()
    brain_activity = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

    agg_dict = {
        'spectrogram_id': 'first',
        'spectrogram_label_offset_seconds': ['min', 'max'],
        'patient_id': 'first',
        'expert_consensus': 'first'
    }

    for tgt in tgt_list:
        agg_dict[tgt] = 'sum'

    groupby = df_csv.groupby(['eeg_id'])
    train = groupby.agg(agg_dict)
    train = train.reset_index()
  
    train.columns = ['eeg_id', 'spectrogram_id', 'min', 'max', 'patient_id', 'target'] + tgt_list
    train['total_votes'] = train[tgt_list].sum(axis=1)
    train[tgt_list] = train[tgt_list].apply(lambda x: x / x.sum(), axis=1)
    
    return train

In [4]:
CREATE_EEGS = False
ALL_EEG_SIGNALS = {}
eeg_paths = list(Path(PATHS.TRAIN_EEGS).glob('*.parquet'))
preload_eegs_path = Path('./inputs/eegs.npy')

if CREATE_EEGS:
    for parquet_path in tqdm(eeg_paths, total=len(eeg_paths)):
        # Save EEG to Python dictionary of numpy arrays
        eeg_id = int(parquet_path.stem)
        eeg_path = str(parquet_path)
        data = eeg_from_parquet(eeg_path, display=eeg_id<1)
        ALL_EEG_SIGNALS[eeg_id] = data

    np.save("./inputs/eegs.npy", ALL_EEG_SIGNALS)
else:
    ALL_EEG_SIGNALS = np.load(preload_eegs_path, allow_pickle=True).item()

In [5]:
train_csv = pd.read_csv(PATHS.TRAIN_CSV)
targets = train_csv.columns[-6:].tolist()

print("targets: ", targets)

train_csv['total_votes'] = train_csv[targets].sum(axis=1)

targets_prob = [f"{t.split('_')[0]}_prob" for t in targets]
train_csv[targets_prob] = train_csv[targets].div(train_csv['total_votes'], axis=0)

train_csv['entropy'] = train_csv[targets_prob].apply(lambda row: sum(rel_entr([1/6]*6, row.values+1e-5)), axis=1)
train_csv['is_hard'] = (train_csv['entropy'] < 5.5).astype(int)

agg_dict = {
    'spectrogram_id': 'first',
    'spectrogram_label_offset_seconds': ['min', 'max'],
    'patient_id': 'first',
    'expert_consensus': 'first',
    'total_votes': 'sum',
    'entropy': 'mean'
}

for tgt in targets:
    agg_dict[tgt] = 'sum'
    
train_all = train_csv.groupby(['eeg_id', 'is_hard']).agg(agg_dict).reset_index()
train_all.columns = ['eeg_id', 'is_hard', 'spectrogram_id', 'min', 'max', 'patient_id', 'target', 'total_votes', 'averge_entropy'] + targets
train_all[targets] = train_all[targets].apply(lambda x: x / x.sum(), axis=1)

print("train_all: ", train_all.shape)
print("hard samples: ", train_all[train_all['is_hard'] == 1].shape)
print("easy samples: ", train_all[train_all['is_hard'] == 0].shape)

train_all.head(15)

targets:  ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
train_all:  (17834, 15)
hard samples:  (3578, 15)
easy samples:  (14256, 15)


,eeg_id,is_hard,spectrogram_id,min,max,patient_id,target,total_votes,averge_entropy,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,1,789577333,0.0,16.0,20654,Other,48,4.584192,0.000000,0.000000,0.250000,0.000000,0.166667,0.583333
1,582999,1,1552638400,0.0,38.0,20230,LPD,154,4.870032,0.000000,0.857143,0.000000,0.071429,0.000000,0.071429
2,642382,0,14960202,1008.0,1032.0,5955,Other,2,7.802343,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,751790,0,618728447,908.0,908.0,38549,GPD,1,7.802343,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
4,778705,0,52296320,0.0,0.0,40955,Other,2,7.802343,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,1629671,0,2036345030,0.0,160.0,37481,Seizure,51,7.802343,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1895581,1,128369999,1138.0,1138.0,47999,Other,13,4.847483,0.076923,0.000000,0.000000,0.000000,0.076923,0.846154
7,2061593,0,320962633,1450.0,1450.0,23828,Other,1,7.802343,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
8,2078097,0,2074135650,3342.0,3342.0,61174,Other,2,7.802343,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
9,2366870,0,1232582129,0.0,30.0,23633,Other,18,6.134196,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667


In [6]:
# ModelConfig.MODEL_BACKBONE = 'transformer'
# ModelConfig.MODEL_NAME = "transformer_1D"
# ModelConfig.BATCH_SIZE = 32
# ModelConfig.GRADIENT_ACCUMULATION_STEPS = 1
# ModelConfig.EPOCHS = 30
# ModelConfig.DROP_RATE = 0.1
# ModelConfig.EARLY_STOP_ROUNDS = 5
# ModelConfig.EEG_SEQ_DOWNSAMPLE = None

# logger = get_logger(PATHS.OUTPUT_DIR, f"{ModelConfig.MODEL_NAME}_train.log")

In [6]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter(order, [lowcut, highcut], fs=fs, btype='band')
    y = lfilter(b, a, data)
    return y

def denoise_filter(x):
    # Sample rate and desired cutoff frequencies (in Hz).
    fs = 200.0
    lowcut = 1.0
    highcut = 25.0
    
    # Filter a noisy signal.
    T = 50
    nsamples = T * fs
    t = np.arange(0, nsamples) / fs
    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=6)
    y = (y + np.roll(y,-1)+ np.roll(y,-2)+ np.roll(y,-3))/4
    y = y[0:-1:4]
    
    return y

def mu_law_encoding(data, mu):
    mu_x = np.sign(data) * np.log(1 + mu * np.abs(data)) / np.log(mu + 1)
    return mu_x

def mu_law_expansion(data, mu):
    s = np.sign(data) * (np.exp(np.abs(data) * np.log(mu + 1)) - 1) / mu
    return s

def quantize_data(data, classes):
    mu_x = mu_law_encoding(data, classes)
    return mu_x #quantized

def butter_lowpass_filter(data, cutoff_freq=20, sampling_rate=200, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_freq / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = lfilter(b, a, data, axis=0)
    return filtered_data

class EEGSeqDataset(Dataset):
    def __init__(self, df, config, eegs, mode='train'):
        self.df = df
        self.config = config
        self.mode = mode
        self.eegs = eegs
        self.downsample = config.EEG_SEQ_DOWNSAMPLE

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        X, y_prob = self.__data_generation(idx)

        if self.downsample is not None:
            X = X[::self.downsample,:]

        return torch.tensor(X, dtype=torch.float32), torch.tensor(y_prob, dtype=torch.float32)
    
    def __data_generation(self, index):
        row = self.df.iloc[index]
        X = np.zeros((10_000, 8), dtype='float32')
        
        data = self.eegs[row.eeg_id]
        # === Feature engineering ===
        X[:,0] = data[:,EEF_FEAT_INDEX['Fp1']] - data[:,EEF_FEAT_INDEX['T3']]
        X[:,1] = data[:,EEF_FEAT_INDEX['T3']] - data[:,EEF_FEAT_INDEX['O1']]

        X[:,2] = data[:,EEF_FEAT_INDEX['Fp1']] - data[:,EEF_FEAT_INDEX['C3']]
        X[:,3] = data[:,EEF_FEAT_INDEX['C3']] - data[:,EEF_FEAT_INDEX['O1']]

        X[:,4] = data[:,EEF_FEAT_INDEX['Fp2']] - data[:,EEF_FEAT_INDEX['C4']]
        X[:,5] = data[:,EEF_FEAT_INDEX['C4']] - data[:,EEF_FEAT_INDEX['O2']]

        X[:,6] = data[:,EEF_FEAT_INDEX['Fp2']] - data[:,EEF_FEAT_INDEX['T4']]
        X[:,7] = data[:,EEF_FEAT_INDEX['T4']] - data[:,EEF_FEAT_INDEX['O2']]

        # === Standarize ===
        X = np.clip(X,-1024, 1024)
        X = np.nan_to_num(X, nan=0) / 32.0

        # === Butter Low-pass Filter ===
        # !!! change to bandpass filter (low=0.5, hight=20, order=2) !!!
        X = butter_lowpass_filter(X)
        # X = butter_bandpass_filter(X, .5, 20, 200, order=2)

        if self.mode != 'test':
            y_prob = row[TARGETS].values.astype(np.float32)
        else:
            y_prob = np.zeros(6, dtype='float32')

        return X, y_prob

In [8]:
# class MultiHeadAttention(nn.Module):
#     def __init__(self, embed_size, heads):
#         super(MultiHeadAttention, self).__init__()
#         self.embed_size = embed_size
#         self.heads = heads
#         self.head_dim = embed_size // heads
        
#         assert (
#             self.head_dim * heads == embed_size
#         ), "Embedding size needs to be divisible by heads"
        
#         self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
#         self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
#         self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
#         self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        
#     def forward(self, values, keys, query):
#         N = query.shape[0]
#         value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]
        
#         # Split the embedding into self.heads pieces
#         values = values.reshape(N, value_len, self.heads, self.head_dim)
#         keys = keys.reshape(N, key_len, self.heads, self.head_dim)
#         queries = query.reshape(N, query_len, self.heads, self.head_dim)
        
#         values = self.values(values)
#         keys = self.keys(keys)
#         queries = self.queries(queries)
        
#         # Attention mechanism
#         energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
#         attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim=3)
        
#         out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
#             N, query_len, self.heads * self.head_dim
#         )
        
#         out = self.fc_out(out)
#         return out

In [23]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Ensure input is in the correct shape (seq_len, batch_size, embed_dim) for nn.MultiheadAttention
        x = x.permute(1, 0, 2)
        
        # Apply multi-head attention
        attention_output, _ = self.attention(x, x, x)
        x = x + self.dropout(attention_output)
        x = self.norm1(x.permute(1, 0, 2))  # Revert to (batch_size, seq_len, embed_dim) for layer normalization
        
        # Apply feed-forward network
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        
        return out
    
class PositionalEmbedding(nn.Module):
    def __init__(self, max_length, embed_size):
        super(PositionalEmbedding, self).__init__()
        self.embed_size = embed_size

        # Create a matrix of shape [max_length, embed_size] representing the positional encoding for max_length positions
        pe = torch.zeros(max_length, embed_size)
        for pos in range(max_length):
            for i in range(0, embed_size, 2):
                pe[pos, i] = np.sin(pos / (10000 ** ((2 * i)/embed_size)))
                pe[pos, i + 1] = np.cos(pos / (10000 ** ((2 * (i + 1))/embed_size)))
                
        pe = pe.unsqueeze(0)  # Add batch dimension [1, max_length, embed_size]
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        # x: Tensor of shape [batch_size, seq_len, embed_size]
        # Adds positional encoding to each sequence element in the batch
        x = x + self.pe[:, :x.size(1)]
        return x
    
class DownsampleBlock(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size, dilation, stride, padding):
        super(DownsampleBlock, self).__init__()
        self.conv = nn.Conv1d(input_channels, output_channels, kernel_size, 
                              stride=stride, padding=padding, dilation=dilation)
        self.activation = nn.ReLU()
        self.norm = nn.BatchNorm1d(output_channels)

    def forward(self, x):
        # x: Tensor of shape [batch_size, channels, seq_len]
        x = self.conv(x)
        x = self.activation(x)
        x = self.norm(x)
        return x


In [31]:
class EEGSeqTransformer(nn.Module):
    def __init__(self, input_channels, seq_length, embed_size, num_transformer_blocks, heads, forward_expansion, dropout, num_classes):
        super(EEGSeqTransformer, self).__init__()
        
        # Initial downsampling and embedding
        self.downsample_block = DownsampleBlock(
            input_channels=input_channels, 
            output_channels=embed_size, 
            kernel_size=3, 
            dilation=4, 
            stride=2, 
            padding=2)
        
        # self.positional_embedding = PositionalEmbedding(seq_length // 2, embed_size)
        
        # # Transformer blocks
        # self.transformer_blocks = nn.Sequential(
        #     *[TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_transformer_blocks)]
        # )
        
        # # Classifier head
        # self.output_linear = nn.Linear(embed_size, num_classes)
        # self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        print("Input shape: ", x.shape)
        # x: [batch_size, seq_length, input_channels]
        x = x.permute(0, 2, 1)  # Conv1D expects [batch_size, channels, length]
        x = self.downsample_block(x)
        print("Downsample output: ", x.shape)

        # x = x.permute(0, 2, 1)  # Back to [batch_size, length, channels] for the rest of the network
        # x = self.positional_embedding(x)
        # print("Positional embedding output: ", x.shape)
        
        # x = self.transformer_blocks(x)
        
        # # Apply the specified pooling type
        # if self.pooling_type == 'average':
        #     x = torch.mean(x, dim=1)
        # elif self.pooling_type == 'sequence':
        #     x = x[:, -1, :]
        # elif self.pooling_type == 'learnable_sequence':
        #     x = self.seq_pool(x)  # Apply learnable sequence pooling
        
        # x = self.dropout(x)
        # x = self.output_linear(x)
        
        return x

In [32]:
train_dataset = EEGSeqDataset(train_all, ModelConfig, ALL_EEG_SIGNALS, mode="train")
train_loader = DataLoader(train_dataset, drop_last=True, batch_size=2, num_workers=4, pin_memory=True, shuffle=False)

model = EEGSeqTransformer(
    input_channels=8, 
    seq_length=10_000, 
    embed_size=512, 
    num_transformer_blocks=1, 
    heads=4, 
    forward_expansion=4, 
    dropout=0.1, 
    num_classes=6
)

model.to(DEVICE)

for batch in train_loader:
    X, y = batch
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    X, y = X.to(DEVICE), y.to(DEVICE)
    # y_pred, resnet_out, rnn_out = model(X)
    y_pred = model(X)
    print(y_pred.shape)
    
    break

X shape: torch.Size([2, 10000, 8])
y shape: torch.Size([2, 6])
Input shape:  torch.Size([2, 10000, 8])
Downsample output:  torch.Size([2, 512, 4998])
Positional embedding output:  torch.Size([2, 4998, 512])


OutOfMemoryError: CUDA out of memory. Tried to allocate 60.00 MiB. GPU 0 has a total capacity of 10.76 GiB of which 9.81 MiB is free. Including non-PyTorch memory, this process has 10.73 GiB memory in use. Of the allocated memory 9.65 GiB is allocated by PyTorch, and 34.94 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)